In [1]:
from pymmcore_plus import CMMCorePlus
from fov import FOV
import useq
from useq import MDAEvent
from queue import Queue
import numpy as np
from matplotlib import pyplot as plt
from napari import Viewer
import numpy as np
from queue import Queue
from pymmcore_plus import CMMCorePlus
from useq import MDAEvent
import useq
from useq._channel import Channel
from MDAEngine_DMD import MDAEngine_DMD
from controller import Controller, Analyzer
import pandas as pd
import random
from utils import ImgType, MetadataDict
from stimulation import StimExtraParameters
from dmd import DMD
from hardware import load_config
import os
import tempfile
from datetime import datetime
import pymmcore
import napari

In [2]:
#TODO check if dmd calibration works
#TODO adjust napari live view, check how they change brightness in napari-umanager (or directly use napari-umanager?)
#TODO add napari widget to store/display positions
#TODO generate necessary folders beforehand
#TODO add to known issues: conda install -c conda-forge llvm-openmp if there are issues with threads on M1

## Load uManager and set default devices

In [2]:
mmc = CMMCorePlus() #Create a new instance of the CMMCorePlus class
mmc.loadSystemConfiguration('local/local_config.cfg')  #Load the system configuration file (standard uManager .cfg file)
load_config(mmc) #Set the default device from the YAML file (specific to this package)

In [3]:
# Create a new log file in the temp directory
log_dir = tempfile.gettempdir()
log_file = os.path.join(log_dir, "log.txt")

# Get the current date and time
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


print(f"Logfile: {log_file}")

# Open the log file in append mode and write the date and time
with open(log_file, "w") as file: #w: write (delete old content), a: append
    file.write(f"{current_time}\n")
    file.write(f"User ID: {mmc.getUserId()}")

    file.write(f"PyMMCore version {pymmcore.__version__}\n")
    file.write(f"{mmc.getAPIVersionInfo()}")
    file.write(f"{mmc.getVersionInfo()}\n\n")

    file.write("Loaded devices:\n")
    file.write("\n".join(str(device) for device in mmc.getLoadedDevices()))
    file.write("\n\n")

Logfile: C:\Users\MICRO-~1\AppData\Local\Temp\log.txt


In [5]:
viewer = napari.Viewer()
image_layer = viewer.add_image(np.ones((1,1)))

In [6]:
viewer.

SyntaxError: invalid syntax (60449377.py, line 1)

In [12]:
pip install "napari[all]"

  Using cached PyQt5-5.15.9-cp37-abi3-win_amd64.whl (6.8 MB)
  Obtaining dependency information for napari-plugin-manager<0.2.0,>=0.1.0a1 from https://files.pythonhosted.org/packages/6a/93/6aba9e00eadd511dc063735c79f5ec806303dcdfbc8249ed5e7695f6a6f0/napari_plugin_manager-0.1.0a2-py3-none-any.whl.metadata
  Obtaining dependency information for PyQt5-sip<13,>=12.11 from https://files.pythonhosted.org/packages/77/04/8513bcf02a02f40575d9639bb09c38ed6a366ac46c4dfd92edde1b922e76/PyQt5_sip-12.12.2-cp39-cp39-win_amd64.whl.metadata
  Using cached PyQt5_Qt5-5.15.2-py3-none-win_amd64.whl (50.1 MB)
   ---------------------------------------- 0.0/78.4 kB ? eta -:--:--
   ------------------------------------ --- 71.7/78.4 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 78.4/78.4 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
viewer.add_image(np.ones((1,1)))

<Image layer 'Image' at 0x26393217910>

## Create list with FOVs

In [83]:
pos_list = [(2,2)]
fovs = []

for i,pos in enumerate(pos_list):
    print(pos)
    fov = FOV(pos=pos,
              index =i,
              path='/tmp/',
              metadata={},
              properties={'stim_property': 'top'},
              )
    fovs.append(fov)

(2, 2)


## Create a DF with all planned acquisitions and stimulations

In [84]:
df_acquire = pd.DataFrame(columns=['fov', 'timestep', 'time', 'treatment', 'acquired','stim', 'channels', 'channels_stim'])

time_between_frames = 5 #time in seconds between frames
stim_timesteps = [2,3]  # list of timesteps
stim_timesteps= []  # list of timesteps

timestep = range(2)  # 0-20
treatments = ['stim_top', 'stim_mid',]  # list of treatments
fovs:list[FOV] = fovs
channels = [['DAPI','FITC']]
channels_stim = ['GFP']

ImgType.IMG_RAW

# Loop over the FOVs and randomly assign one of the treatments to it
treatments_shuffled = treatments.copy()
random.shuffle(treatments_shuffled)
for fov in fovs:
    treatment = treatments_shuffled[fov.index % len(treatments_shuffled)]
    for timestep in timestep:
        new_row = {'fov_object': fov,
                    'fov':fov.index,
                    'timestep': timestep,
                    'time': timestep*time_between_frames,
                    'treatment': treatment,
                    'acquired': False,
                    'stim': False,
                    'channels': channels,
                    'channels_stim' : channels_stim,
                    'fname' : f'{str(fov.index).zfill(3)}_{str(timestep).zfill(5)}',

                    }
        df_acquire = pd.concat([df_acquire, pd.DataFrame(new_row, index=[0])])

for timestep in stim_timesteps:
    df_acquire.loc[df_acquire['timestep'] == timestep, 'stim'] = True
df_acquire = df_acquire.sort_values(by=['timestep', 'fov'])

df_acquire

,fov,timestep,time,treatment,acquired,stim,channels,channels_stim,fov_object,fname
0,0,0,0,stim_mid,False,False,"[DAPI, FITC]",GFP,<fov.FOV object at 0x2d7108e80>,000_00000
0,0,1,5,stim_mid,False,False,"[DAPI, FITC]",GFP,<fov.FOV object at 0x2d7108e80>,000_00001


## Take an image at the current location

In [89]:
print(f'Channels to acquire: {channels[0]}')

Channels to acquire: ['DAPI', 'FITC']


## Run on virtual system 

In [86]:
from add_frame import ImageProcessingPipeline
from segmentation import SegmentatorStardist
from pymmcore_plus.mda import MDAEngine
from stimulation import StimExtraParameters
from controller import Analyzer

segmentator = SegmentatorStardist('2D_versatile_fluo')
stimulator = StimExtraParameters()
pipeline = ImageProcessingPipeline(segmentator,stimulator)
analyzer = Analyzer(pipeline)


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


TypeError: __init__() missing 1 required positional argument: 'tracker'

In [ ]:
mmc = CMMCorePlus()

mmc.loadSystemConfiguration()

#setup camera
mmc.initializeDevice("Camera");
mmc.setCameraDevice("Camera");

# Register the custom engine with the runner
dmd = DMD(mmc, test_mode = True)
mda_engine_dmd = MDAEngine_DMD(dmd)

mmc.mda.set_engine(mda_engine_dmd)

queue = Queue()
controller = Controller(analyzer, mmc, queue)

# Start the acquisition
controller.run(df_acquire)
#controller.run_test()


2023-08-25 13:07:33.722 | INFO     | pymmcore_plus.mda._runner:_prepare_to_run:191 - MDA Started: <callable_iterator object at 0x29a892ee0>


Current timestep: 0
  fov timestep time treatment acquired   stim      channels channels_stim   
0   0        0    0  stim_top    False  False  [DAPI, FITC]           GFP  \

                        fov_object  
0  <FOV.FOV object at 0x104c72cd0>  

Current timestep: 1
  fov timestep time treatment acquired   stim      channels channels_stim   
0   0        1    5  stim_top    False  False  [DAPI, FITC]           GFP  \

                        fov_object  
0  <FOV.FOV object at 0x104c72cd0>  



2023-08-25 13:07:33.728 | INFO     | pymmcore_plus.mda._runner:run:148 - index=mappingproxy({'p': 0, 'c': 0}) channel=Channel(config='DAPI') x_pos=2.0 y_pos=2.0


2023-08-25 13:07:33.740 | INFO     | pymmcore_plus.mda._runner:run:148 - index=mappingproxy({'p': 0, 'c': 1}) channel=Channel(config='FITC') x_pos=2.0 y_pos=2.0
2023-08-25 13:07:33.753 | INFO     | pymmcore_plus.mda._runner:run:148 - index=mappingproxy({'p': 0, 'c': 0}) channel=Channel(config='DAPI') x_pos=2.0 y_pos=2.0
base.py (395): Predicting on non-float input... ( forgot to normalize? )
2023-08-25 13:07:33.765 | INFO     | pymmcore_plus.mda._runner:run:148 - index=mappingproxy({'p': 0, 'c': 1}) channel=Channel(config='FITC') x_pos=2.0 y_pos=2.0
2023-08-25 13:07:33.781 | INFO     | pymmcore_plus.mda._runner:run:148 - index=mappingproxy({'p': 0, 'c': 0}) channel=Channel(config='DAPI') x_pos=2.0 y_pos=2.0
2023-08-25 13:07:33.801 | INFO     | pymmcore_plus.mda._runner:run:148 - index=mappingproxy({'p': 0, 'c': 1}) channel=Channel(config='FITC') x_pos=2.0 y_pos=2.0
2023-08-25 13:07:33.824 | INFO     | pymmcore_plus.mda._runner:run:148 - index=mappingproxy({'p': 0, 'c': 0}) channel=Chan

done
Frame {'p': 0, 'c': 0} received: (512, 512)
done
Frame {'p': 0, 'c': 1} received: (512, 512)
Last channel. Sending to _analyzer: (2, 512, 512)
Received image
{'fov': <FOV.FOV object at 0x104c72cd0>, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': 'FITC', 'timestep': 0}
Received raw image
(512, 512)
done
Frame {'p': 0, 'c': 0} received: (512, 512)
done
Frame {'p': 0, 'c': 1} received: (512, 512)
Last channel. Sending to _analyzer: (2, 512, 512)
Received image
{'fov': <FOV.FOV object at 0x104c72cd0>, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': 'FITC', 'timestep': 1}
Received raw image
(512, 512)
done
Frame {'p': 0, 'c': 0} received: (512, 512)
done
Frame {'p': 0, 'c': 1} received: (512, 512)
Last channel. Sending to _analyzer: (2, 512, 512)
Received image
{'fov': <FOV.FOV object at 0x104c72cd0>, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': 'FITC', 'timestep': 0}
Received raw image
(512, 512)
done
Frame {'p': 0, 'c': 0} received: (512, 512)
done
Frame {'p': 0, 'c': 1} re

In [11]:
configs = mmc.getAvailableConfigs('Channel')
for config in configs:
    print(config)


Cy5
DAPI
FITC
Rhodamine


In [6]:
assert np.array_equal(segmentator.segment(np.zeros((100,100))), np.zeros((100,100)))
assert np.array_equal(stimulator.get_stim_mask(np.zeros((10,10)),FOV(0,0,'/tmp/',{},{'stim_property': 'top'})),np.ones((10,10)))

(100, 100)


In [8]:
from add_frame import store_img
fov = FOV(pos=(0,0),index =i, path='/tmp/', metadata={'stim_property': 'top'},properties={},)
metadata = {'fov':0,'img_type':ImgType.IMG_RAW,'fname':'test', 'fov_object':fov}
store_img(np.ones((4,10,10)),metadata,'/tmp/')

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/luhin/miniforge3/envs/micro-llm/lib/python3.9/threading.py", line 950, in _bootstrap_inner
    self.run()
  File "/Users/luhin/miniforge3/envs/micro-llm/lib/python3.9/threading.py", line 888, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/luhin/sync/phd/code/rtm/add_frame.py", line 41, in run
    stim_mask = self.stimulator.get_stim_mask(segmented)
TypeError: get_stim_mask() missing 1 required positional argument: 'fov'


: 

: 

In [ ]:
mmc.snapImage()
mmc.getImage()
mmc.setExposure(100)
mmc.getCameraDevice()

'Camera'

## Run on system with DMD

In [ ]:
from add_frame import ImageProcessingPipeline
from segmentation import SegmentatorStardist

segmentator = SegmentatorStardist('2D_versatile_fluo')
stimulator = StimExtraParameters()
pipeline = ImageProcessingPipeline(segmentator,stimulator)
analyzer = Analyzer(pipeline)



mmc = CMMCorePlus()
dmd = DMD(mmc)

mmc.loadSystemConfiguration()
mmc.mda.set_engine(MDAEngine_DMD(mmc,dmd))

# Apply the custom acquisition engine
# Register the custom engine with the runner
core = CMMCorePlus.instance()
#core.mda.set_engine(MDAEngine_DMD(core))

# create the Queue that will hold the MDAEvents
queue = Queue()



controller = Controller(analyzer, mmc , queue, dmd)
# Start the acquisition
controller.run()

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


RuntimeError: No device with label ""

In [ ]:
"""Simple simulator demonstrating event-driven acquisitions with pymmcore-plus."""
import random
import time
from queue import Queue

import numpy as np
from pymmcore_plus import CMMCorePlus
from useq import MDAEvent, MDASequence


class Analyzer:
    """Analyzes images and returns a dict of results."""

    def run(self, data) -> dict:
        pass
        # Fake analysis; randomly return a dict with a value of None 10% of the time
        #if random.random() < 0.1:
        #return {"result": "STOP"}

       # else:
        #return {"result": random.random()}


class Controller:
    STOP_EVENT = object()

    def __init__(self, analyzer: Analyzer, mmc: CMMCorePlus, queue: Queue):
        self._analyzer = analyzer  # analyzer of images
        self._queue = queue  # queue of MDAEvents
        self._results: dict = {}  # results of analysis

        self._mmc = mmc
        mmc.mda.events.frameReady.connect(self._on_frame_ready)

    def _on_frame_ready(self, img: np.ndarray, event: MDAEvent) -> None:
        # Analyze the image
        #print(event)
        self._results = self._analyzer.run(img)
        print(event.metadata['fov'])

    def run(self) -> None:
        # convert the queue to an iterable
        queue_sequence = iter(self._queue.get, self.STOP_EVENT)

        # Start the acquisition (run_mda is non-blocking)
        self._mmc.run_mda(queue_sequence)

        # Queue the first image acquisition
## Create MDASequence with a time delay of 1 second
        
        interval = 2
        nb_fovs = 2
        delay = interval/nb_fovs

        mda_sequence_fov_0 = MDASequence(delay=10)
        mda_sequence_fov_1 = MDASequence(exposure = 10)

        channel = {'config': 'Cy5', 'exposure': 10}

        for frame in range(3):
            self._queue.put(MDAEvent(exposure=10,mda_sequence=mda_sequence_fov_0, min_start_time=frame *interval,metadata={'fov':0},channel=channel))
            self._queue.put(MDAEvent(exposure=10,mda_sequence=mda_sequence_fov_1, min_start_time=frame *interval+delay,metadata={'fov':1}))


       # self._queue.put(intitial_event)
        self._queue.put(self.STOP_EVENT)


# Setup the MM Core
mmc = CMMCorePlus()
mmc.loadSystemConfiguration()

# create the Queue that will hold the MDAEvents
q = Queue()

# Setup the controller and analyzer
analyzer = Analyzer()
controller = Controller(analyzer, mmc, q)

# Start the acquisition
controller.run()





2023-08-30 12:55:11.051 | INFO     | pymmcore_plus.mda._runner:_prepare_to_run:221 - MDA Started: GeneratorMDASequence()
2023-08-30 12:55:11.052 | INFO     | pymmcore_plus.mda._runner:_run:186 - channel=Channel(config='Cy5') exposure=10.0 min_start_time=0.0 metadata={'fov': 0}


0


2023-08-30 12:55:12.055 | INFO     | pymmcore_plus.mda._runner:_run:186 - exposure=10.0 min_start_time=1.0 metadata={'fov': 1}


1


2023-08-30 12:55:13.057 | INFO     | pymmcore_plus.mda._runner:_run:186 - channel=Channel(config='Cy5') exposure=10.0 min_start_time=2.0 metadata={'fov': 0}


...

1


2023-08-30 12:55:15.057 | INFO     | pymmcore_plus.mda._runner:_run:186 - channel=Channel(config='Cy5') exposure=10.0 min_start_time=4.0 metadata={'fov': 0}


0


2023-08-30 12:55:16.057 | INFO     | pymmcore_plus.mda._runner:_run:186 - exposure=10.0 min_start_time=5.0 metadata={'fov': 1}
2023-08-30 12:55:16.074 | INFO     | pymmcore_plus.mda._runner:_finish_run:313 - MDA Finished: GeneratorMDASequence()


1


In [10]:
def normalize_mi_ma(x, mi, ma, clip=False, eps=1e-20, dtype=np.float32):
    if dtype is not None:
        x   = x.astype(dtype,copy=False)
        mi  = dtype(mi) if np.isscalar(mi) else mi.astype(dtype,copy=False)
        ma  = dtype(ma) if np.isscalar(ma) else ma.astype(dtype,copy=False)
        eps = dtype(eps)

def normalize(x, pmin=3, pmax=99.8, axis=None, clip=False, eps=1e-20, dtype=np.float32):
    """Percentile-based image normalization."""

    mi = np.percentile(x.flatten(),pmin)
    ma = np.percentile(x.flatten(),pmax)
    return normalize_mi_ma(x, mi, ma, clip=clip, eps=eps, dtype=dtype)

normalize(np.ones((1000,1000)))
print(np.ones((1000,1000)).flatten().shape)

(1000000,)


In [2]:
import random
import time
from queue import Queue

import numpy as np
from pymmcore_plus import CMMCorePlus
from useq import MDAEvent, MDASequence


In [4]:
mda_sequence_2 = MDASequence(exposure = 10)
for event in mda_sequence_2:
    intitial_event = event
    print('here') 

In [2]:
import napari
viewer = napari.Viewer()

In [4]:
import napari_umanager
from napari_umanager import napari_manager
napari_manager(viewer, mmc, q)

ModuleNotFoundError: No module named 'napari_umanager'